In [1]:
from models import *
from strategies import *
from datasets import *
import numpy as np

import tqdm
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import os
import torch
import torch.nn as nn

from PIL import Image
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset

In [2]:
params = {'n_epoch': 50, 
               'train_args':{'batch_size': 1, 'num_workers': 0},
               'test_args':{'batch_size': 1, 'num_workers': 0},
               'optimizer_args':{'lr': 0.00005, 'momentum': 0.5}}

In [3]:
def get_chorus_data(handler):
    raw_train = Chorus_dataset("processed/", split='train', init=True, transform=True)
    raw_test = Chorus_dataset("processed/", split='test', init=True, transform=True)
    return Data(raw_train.images, raw_train.labels, raw_test.images, raw_test.labels, handler)


In [4]:

model = smp.create_model(
            'FPN', encoder_name='resnet34', in_channels=3, classes = 1
        )
net = Net(model, params, device = torch.device("cuda:0"))
data = get_chorus_data(Handler)
data.initialize_labels(5)



### Choose an AL strategy from a)RandomSampling b)MarginSampling c)EntropySampling d)KCenterGreedy e)AdversarialBIM

In [5]:
strategy = RandomSampling(data, net)


In [6]:
print("Round 0")
strategy.train()
logits, mask_gt = strategy.predict(data.get_test_data())
print(f"Round 0 testing metrics: {data.cal_test_metrics(logits, mask_gt )}")

for rd in range(1, 13):
    print(f"Round {rd}")

    # query
    query_idxs = strategy.query(5)

    # update labels
    strategy.update(query_idxs)
    strategy.train()

    # calculate accuracy
    logits, maks_gt = strategy.predict(data.get_test_data())
    print(f"Round {rd} testing metrics: {data.cal_test_metrics(logits, mask_gt)}")

Round 0


100%|███████████████████████████████████████████████████████████████| 50/50 [00:31<00:00,  1.60it/s]


Round 0 testing metrics: (tensor(0.0754), tensor(0.1402))
Round 1


100%|███████████████████████████████████████████████████████████████| 50/50 [01:02<00:00,  1.24s/it]


Round 1 testing metrics: (tensor(0.0831), tensor(0.1534))
Round 2


...

Round 10 testing metrics: (tensor(0.5549), tensor(0.7138))
Round 11


100%|███████████████████████████████████████████████████████████████| 50/50 [05:49<00:00,  7.00s/it]


Round 11 testing metrics: (tensor(0.5493), tensor(0.7091))
Round 12


100%|███████████████████████████████████████████████████████████████| 50/50 [06:20<00:00,  7.61s/it]


Round 12 testing metrics: (tensor(0.5509), tensor(0.7104))


In [6]:

#Margin Sampling
print("Round 0")
strategy.train()
logits, mask_gt = strategy.predict(data.get_test_data())
print(f"Round 0 testing metrics: {data.cal_test_metrics(logits, mask_gt )}")

for rd in range(1, 13):
    print(f"Round {rd}")

    # query
    query_idxs = strategy.query(5)

    # update labels
    strategy.update(query_idxs)
    strategy.train()

    # calculate accuracy
    logits, maks_gt = strategy.predict(data.get_test_data())
    print(f"Round {rd} testing metrics: {data.cal_test_metrics(logits, mask_gt)}")

Round 0


100%|███████████████████████████████████████████████████████████████| 50/50 [00:33<00:00,  1.48it/s]


Round 0 testing metrics: (tensor(0.0733), tensor(0.1365))
Round 1


100%|███████████████████████████████████████████████████████████████| 50/50 [01:04<00:00,  1.29s/it]


Round 1 testing metrics: (tensor(0.0751), tensor(0.1397))
Round 2


...

Round 10 testing metrics: (tensor(0.5007), tensor(0.6673))
Round 11


100%|███████████████████████████████████████████████████████████████| 50/50 [06:00<00:00,  7.22s/it]


Round 11 testing metrics: (tensor(0.5054), tensor(0.6714))
Round 12


100%|███████████████████████████████████████████████████████████████| 50/50 [06:18<00:00,  7.58s/it]


Round 12 testing metrics: (tensor(0.5203), tensor(0.6845))
